# Generating recordings integer recordings

This notebook shows how to generate recordings that mimic an acquisition system with a certain ADC bit depth (number of bits used by the Analog to Digital Converter) and LSB (least significant bit) or gain to uV.

With MEArec, when selecting an `int` dtype, we can specify the number of bits needed to encode the input signal or the gain to convert the integer values to uV.

In addition, we can also specify the value of the LSB in the output recording.


The ADC bit depth (if gain is not given) and LSB controls the final range of the signal:

```
signal_range = lsb * (2^adc_bit_depth)
```

If `gain` is provided, this is intended as the final gain to recover uV values. Since MEArec 
already generates uV, the the signal range becomes:

```
signal_range = lsb / gain
```


Note that if LSB and ADC bit depth are too large they might exceed the value ranges allowed by the dtype (e.g. `int16`). In this case, you can try to define a *deeper* dtype (e.g. `int32`).

The final recording has an attribute called `gain_to_uV`, which is also saved to file.

In [ ]:
import MEArec as mr
import MEAutility as mu
import yaml
import numpy as np
from pprint import pprint
import matplotlib.pylab as plt
%matplotlib notebook

## Load default configuration files

First, let's load the default configuration of MEArec

In [ ]:
default_info, mearec_home = mr.get_default_config()
pprint(default_info)

## Load pre-generated templates

Templates are loaded from the `generate_templates_and_recordings.ipynb` notebook:

In [ ]:
tempgen = mr.load_templates('data/test_templates.h5')

## Generating and saving recordings with different ADC depths and LSBs

We assume that the `data/test_templates.h5` templates has already been generated (run `generate_templates_and_recordings.ipynb` notebook first).

In [ ]:
recordings_params = mr.get_default_recordings_params()
pprint(recordings_params)

Similarly to the templates generation, we can change th eparameters that we pass to the `gen_recordings` function.
In this case we will keep the default parameters.

In [ ]:
adc_bit_depths = [10, 12]
lsbs = [1, 4, 8]
gains = [0.195, 0.3]

In [ ]:
# set seeds for reproducibility
recordings_params["seeds"]["templates"] = 0
recordings_params["seeds"]["spiketrains"] = 1
recordings_params["seeds"]["convolution"] = 2
recordings_params["seeds"]["noise"] = 3

recordings_params["recordings"]["dtype"] = "int16"
recordings_params["recordings"]["filter"] = False

In [ ]:
recgen_dict = {}
for adc_bit_depth in adc_bit_depths:
    for lsb in lsbs:
        print(f"\n\nADC DEPTH: {adc_bit_depth} -- LSB: {lsb}\n\n")
        recordings_params["recordings"]["adc_bit_depth"] = adc_bit_depth
        recordings_params["recordings"]["lsb"] = lsb

        recgen = mr.gen_recordings(templates='data/test_templates.h5', params=recordings_params,
                                   verbose=False)
        recgen_dict[f"adc{adc_bit_depth}_lsb{lsb}"] = recgen
        print(f"Recording gain: {recgen.gain_to_uV}")
        
# use gain instead
for gain in gains:
    for lsb in lsbs:
        print(f"\n\nGAIN: {gain} -- LSB: {lsb}\n\n")
        recordings_params["recordings"]["gain"] = gain
        recordings_params["recordings"]["lsb"] = lsb

        recgen = mr.gen_recordings(templates='data/test_templates.h5', params=recordings_params,
                                   verbose=False)
        recgen_dict[f"gain{gain}_lsb{lsb}"] = recgen
        print(f"Recording gain: {recgen.gain_to_uV}")

In [ ]:
ch = 10
fig, axs = plt.subplots(nrows=2)

for i, (rec_name, recgen) in enumerate(recgen_dict.items()):
    axs[0].plot(recgen.recordings[:30000, ch], color=f"C{i}", alpha=0.7, label=rec_name)
    axs[0].set_title("Unscaled")
    axs[1].plot(recgen.recordings[:30000, ch] * recgen.gain_to_uV, color=f"C{i}", alpha=0.7, label=rec_name)
    axs[1].set_title("Scaled")
    
axs[0].legend()